# From Datanode graphs to Data Journeys

In [31]:
# Parameters
indir = "./rdf"
outdir = "./datajourneys"
didir = "./graphs"
classifier = "MLPClassifier_2_1000_rdf2vec.clf"# "LogisticRegression_2_200_rdf2vec.clf"#"MLPClassifier_2_200_rdf2vec.clf"
rdfgraph = "rdfgraph_2_1000_rdf2vec.ttl" #"rdfgraph_2_200_rdf2vec.ttl"
notebook = 'very-simple-pytorch-training-0-59' #'transfer-learning' #'very-simple-pytorch-training-0-59' #'1-quick-start-read-csv-and-flatten-json-fields' 
rdf_folder = './rdf'
clf_folder = './models'
use_rules = True

In [32]:
# notebook = notebook[:-6]
# print(notebook)

very-simple-pytorch-training-0-59


In [17]:
import pandas as pd
import numpy as np
import csv
import json

from os import listdir
from os.path import isfile, join
#from torch.utils.tensorboard import SummaryWriter
import datajourney as DJ
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, BNode, Literal

import networkx.drawing, networkx.drawing.nx_agraph as ag

from pyrdf2vec import RDF2VecTransformer
from pyrdf2vec.embedders import Word2Vec
from pyrdf2vec.graphs import KG, Vertex
from pyrdf2vec.walkers import RandomWalker, Walker

# import pygraphviz
from graphviz import Source

from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

# import sklearn.metrics
# from sklearn import tree
# from sklearn import linear_model
# from sklearn.model_selection import train_test_split
#
# from transformers import AutoTokenizer, AutoModelForMaskedLM

import pickle

In [18]:
def create_kg(graph, label_predicates):
    kg = KG(location='emptygraph.ttl')
    for (s, p, o) in graph:
        if p not in label_predicates:
            s_v = Vertex(str(s))
            o_v = Vertex(str(o))
            p_v = Vertex(str(p), predicate=True, vprev=s_v, vnext=o_v)
            kg.add_vertex(s_v)
            kg.add_vertex(p_v)
            kg.add_vertex(o_v)
            kg.add_edge(s_v, p_v)
            kg.add_edge(p_v, o_v)
    return kg

In [19]:
# Load RDF notebook
rdf_nb = rdflib.Graph()
rdf_nb.parse(indir + '/' + notebook + '.ttl' , format="ttl")
# Load general graph and add notebook
rdf_graph = rdflib.Graph()
rdf_graph.parse(clf_folder + "/" + rdfgraph, format="ttl")
rdf_graph.parse(indir + '/' + notebook + '.ttl' , format="ttl")
# Remove rdfs labels before creating the KG
label_predicates = ["http://www.w3.org/2000/01/rdf-schema#label"] #skip labels
kg = create_kg(rdf_graph, label_predicates)


In [20]:
# Construct activity annotation with rules
if use_rules:
    with open('activity_rules.json') as json_file:
        data = json.load(json_file)
        for q in data:
            #print('Query: ' + q)
            for t in rdf_nb.query(q):
                print("Adding", t)
                rdf_nb.add(t)


Adding (rdflib.term.URIRef('http://purl.org/dj/python/lib/451478648'), rdflib.term.URIRef('http://purl.org/datajourneys/hasActivity'), rdflib.term.URIRef('http://purl.org/datajourneys/Reuse'))
Adding (rdflib.term.URIRef('http://purl.org/dj/python/lib/247464747'), rdflib.term.URIRef('http://purl.org/datajourneys/hasActivity'), rdflib.term.URIRef('http://purl.org/datajourneys/Reuse'))
Adding (rdflib.term.URIRef('http://purl.org/dj/python/lib/278659940'), rdflib.term.URIRef('http://purl.org/datajourneys/hasActivity'), rdflib.term.URIRef('http://purl.org/datajourneys/Reuse'))
Adding (rdflib.term.URIRef('http://purl.org/dj/python/lib/92013028'), rdflib.term.URIRef('http://purl.org/datajourneys/hasActivity'), rdflib.term.URIRef('http://purl.org/datajourneys/Reuse'))
Adding (rdflib.term.URIRef('http://purl.org/dj/python/lib/73138615'), rdflib.term.URIRef('http://purl.org/datajourneys/hasActivity'), rdflib.term.URIRef('http://purl.org/datajourneys/Reuse'))
Adding (rdflib.term.URIRef('http://pu

In [21]:
# Select nodes
djo = "http://purl.org/datajourneys/"
entities = []
for s, p, o in rdf_nb.triples((None, None, None)):
    # check if has already an activity
    a = rdf_nb.value(s, URIRef(djo + "hasActivity"))
    if a == None:
        entities.append(s)
    if (type(o).__name__ == 'URIRef' and p != RDF.type and p != URIRef(djo + "hasActivity")):
        a = rdf_nb.value(o, URIRef(djo + "hasActivity"))
        if a == None:
            entities.append(o)
entities = list(set(entities))
# Remove the root
root = URIRef('http://purl.org/dj/kaggle/' + notebook)
# typenb = URIRef('http://purl.org/dj/kaggle/Notebook')
entities.remove(root)
# entities.remove(typenb)
entities

[rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#635569328'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#29950131'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#774178053'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#314245853'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#731841850'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#431162393'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#84214120'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#172425776'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#390398790'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#389284681'),
 rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pyt

In [22]:
# Build embeddings
random_walker = RandomWalker(10, 100) #float('inf')
transformer = RDF2VecTransformer(Word2Vec(sg=0),walkers=[random_walker])
predict = transformer.fit_transform(kg, entities)

In [23]:
# Classifier
clf = pickle.load(open(clf_folder + '/' + classifier, 'rb'))

In [24]:
prediction = clf.predict(predict)

In [25]:
activities = list(zip(entities,prediction))
activities

[(rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#635569328'),
  ':Reuse'),
 (rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#29950131'),
  ':Reuse'),
 (rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#774178053'),
  ':Reuse'),
 (rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#314245853'),
  ':Reuse'),
 (rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#731841850'),
  ':Preparation'),
 (rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#431162393'),
  ':Reuse'),
 (rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#84214120'),
  ':Reuse'),
 (rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#172425776'),
  ':Reuse'),
 (rdflib.term.URIRef('http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#390398790'),
  ':Reuse'),
 (rdflib.term.URIRef('ht

In [26]:
dn_annotated = rdf_nb
djo = "http://purl.org/datajourneys/"
for (entity, activity) in activities:
    dn_annotated.add((entity, URIRef(djo + "hasActivity") , URIRef(djo + activity[1:])))

for(s,p,o) in dn_annotated:
    print(s,p,o)

http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#3493925377 http://purl.org/dj/appearsIn http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59
http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#172163630 http://www.w3.org/2000/01/rdf-schema#label self(2)
http://purl.org/dj/python/lib/30474470 http://purl.org/datajourneys/hasActivity http://purl.org/datajourneys/Reuse
http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#165478946 http://purl.org/dj/appearsIn http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59
http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#460983610 http://www.w3.org/2000/01/rdf-schema#label Training complete in {:.0f}m {:.0f}s(0)
http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#253166210 http://purl.org/dj/read_csv http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#172163630
http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#635569328 http://purl.org/datajourneys/hasActivity http://purl.o

In [27]:
# Find starting nodes (o that are never s)
# ol = []
# for (s,p,o) in dn_annotated.triples((None, None, None)):
#     if(type(o).__name__ == 'URIRef' and p != RDF.type):
#         ol.append(o)
# ol = list(set(ol))
# for oo in ol:
#     if (oo, None, None) in dn_annotated:
#         print(oo, True)
#         ol.remove(oo)
#     else:
#         print(oo,False)
# ol

# qres = dn_annotated.query(
#     """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# SELECT ?s ?p ?o 
#        WHERE {
#           ?s ?p ?o
#           filter (?s = <http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59> || ?o = <http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59>)
#        }""")
# for row in qres:
#     print("%s %s %s" % row)
def n2l(n, g):
    if n == None:
        return "None"
    na = getActivity(n, g)
    delim = '/'
    if '#' in str(n):
        delim = '#'
    #print(delim, n, g.label(n), na)
    nal = ""
    if na != None:
        nal = " (" + g.label(n) + ": " + str(na).rsplit('/', 1)[1] + ")"
    return n.rsplit(delim, 1)[1] + nal

def getActivity(thisNode, graph):
    if(thisNode == None):
        return None
    REUSE = URIRef("http://purl.org/datajourneys/Reuse")
    MOVEMENT = URIRef("http://purl.org/datajourneys/Movement")
    ANALYSIS = URIRef("http://purl.org/datajourneys/Analysis")
    VISUALISATION = URIRef("http://purl.org/datajourneys/Visualisation")
    PREPARATION = URIRef("http://purl.org/datajourneys/Preparation")    
    motifs = [REUSE,ANALYSIS,MOVEMENT,VISUALISATION,PREPARATION]
    activities = graph.objects(thisNode, URIRef(djo + "hasActivity"))
    for activity in activities:
        if activity in motifs:
            return activity
    return None


def walkActivities(lastActivityNode, thisNode, graph, d, visited):

    if thisNode in visited:
        path = ""
        for v in visited:
            path = "".join([path , "/" , n2l(v, graph)])
        print(path + n2l(thisNode, graph) + "[" + str(d) + "]")
        return graph
    
    visited.append(thisNode)
    #print(n2l(lastActivityNode, graph), " >>> ", n2l(thisNode, graph), " - ", d)
    d=d+1
    #print("Walking %s (%s)" % (thisNode, d))
    thisActivity = getActivity(thisNode, graph)
    lastActivity = getActivity(lastActivityNode, graph)
    nextNodes = graph.subjects(None, thisNode)
    #     for n in nextNodes:
    #         print(">",n2l(thisNode, graph), ">>", n2l(n,graph))
    #     print("---")
    if(thisActivity == None):
        # root here
        for n in graph.subjects(None, thisNode):
            graph.add((n, URIRef("http://purl.org/datajourneys/previousActivity"), thisNode))
            graph = walkActivities(thisNode, n, graph, d, visited.copy())
    elif(thisActivity == lastActivity):
        for n in graph.subjects(None, thisNode):
            graph = walkActivities(lastActivityNode, n, graph, d, visited.copy())
    else:
        if(lastActivity != None):
            graph.add((thisNode, URIRef("http://purl.org/datajourneys/previousActivity"), lastActivityNode))
        for n in graph.subjects(None, thisNode):
            graph = walkActivities(thisNode, n, graph, d, visited.copy())
    return graph
        

datajourney = dn_annotated
#http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#525206600 http://purl.org/dj/appearsIn http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59
#print(getActivity(URIRef("http://purl.org/dj/kaggle/very-simple-pytorch-training-0-59#525206600"), dn_annotated))
rootNode = URIRef('http://purl.org/dj/kaggle/' + notebook)
datajourney = walkActivities(None, rootNode, datajourney, 0, [])


/very-simple-pytorch-training-0-59/3493925377 (../input/aptos2019-blindness-detection/train.csv(0): Reuse)/1127482937 (train_dataset(0)$0: Reuse)/858064198 (data_loader(0)$0: Reuse)/170918373 (tk0(0)$0: Preparation)/43319526 (d(0): Preparation)/389284681 (labels(0)$0: Preparation)389284681 (labels(0)$0: Preparation)[7]
/very-simple-pytorch-training-0-59/165478946 (True(0): Reuse)/858064198 (data_loader(0)$0: Reuse)/170918373 (tk0(0)$0: Preparation)/43319526 (d(0): Preparation)/389284681 (labels(0)$0: Preparation)389284681 (labels(0)$0: Preparation)[6]
/very-simple-pytorch-training-0-59/525206600 (enumerate(0): Reuse)/43319526 (d(0): Preparation)/389284681 (labels(0)$0: Preparation)389284681 (labels(0)$0: Preparation)[4]
/very-simple-pytorch-training-0-59/121635265 (len(0): Reuse)/170918373 (tk0(0)$0: Preparation)/43319526 (d(0): Preparation)/389284681 (labels(0)$0: Preparation)389284681 (labels(0)$0: Preparation)[5]
/very-simple-pytorch-training-0-59/43319526 (d(0): Preparation)/389284

In [28]:
# djg = datajourney.triples((None, URIRef("http://purl.org/datajourneys/previousActivity"), None))
# for (s,p,o) in djg:
#     sa = getActivity(s, datajourney)
#     oa = getActivity(o, datajourney)
#     print(s,sa,o,oa)

In [29]:

def buildDjDiGraph(g):
    gd = networkx.DiGraph()
    for (s,p,o) in g.triples((None, URIRef("http://purl.org/datajourneys/previousActivity"), None)):
        gd.add_node(s,label=n2l(s,g))
        gd.add_node(o,label=n2l(o,g))
        gd.add_edge(s,o,label="previous")
    return gd

def buildDnDigraph(g):
    gd = networkx.DiGraph()
    for (s,p,o) in g.triples((None, None, None)):
        if type(o) == rdflib.term.URIRef and p != RDF.type and p != URIRef(djo + "hasActivity"):
            gd.add_node(s,label=n2l(s,g))
            gd.add_node(o,label=n2l(o,g))
            gd.add_edge(s,o,label=n2l(p, g))
    return gd
    
def saveDjDiGraph(fname, gd):
    tag = ag.to_agraph(gd)
    o = open(outdir + "/" + fname + "_DJ.digraph", "w")
    o.write(tag.string())
    
def saveDnDiGraph(fname, gd):
    tag = ag.to_agraph(gd)
    o = open(outdir + "/" + fname + "_DN.digraph", "w")
    o.write(tag.string())

def buildDjPNG(fname):
    src = Source.from_file(outdir + "/" + fname + "_DJ.digraph")
    src.format = "png"
    src.render(outdir + "/" + fname + "_DJ")

def buildDnPNG(fname):
    src = Source.from_file(outdir + "/" + fname + "_DN.digraph")
    src.format = "png"
    src.render(outdir + "/" + fname + "_DN")

def buildDjSVG(fname):
    src = Source.from_file(outdir + "/" + fname + "_DJ.digraph")
    src.format = "svg"
    src.render(outdir + "/" + fname + "_DJ")

def buildDnSVG(fname):
    src = Source.from_file(outdir + "/" + fname + "_DN.digraph")
    src.format = "svg"
    src.render(outdir + "/" + fname + "_DN")

def buildOriginalPNG(fname):
    src = Source.from_file(didir + "/" + fname + ".digraph")
    src.format = "png"
    src.render(outdir + "/" + fname)

In [30]:
dn = buildDnDigraph(rdf_nb)
saveDnDiGraph(notebook, dn)
buildDnPNG(notebook)
buildDnSVG(notebook)

gd = buildDjDiGraph(datajourney)
saveDjDiGraph(notebook, gd)
buildDjPNG(notebook)
buildDjSVG(notebook)

buildOriginalPNG(notebook)
